<a href="https://colab.research.google.com/github/VishwasDevnani/Pneumonia/blob/main/98%25accuracy_chest_x_ray_images_pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas
import os
import glob
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import keras
import os
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential

import seaborn as sns
import glob

import shutil

from shutil import unpack_archive
from subprocess import check_output

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b0c242cf-4c57-6825-58e8-f9b48a441473)


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [6]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [7]:
pwd

'/content/gdrive/My Drive/Kaggle'

In [8]:
# ! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [9]:
# !unzip \*.zip  && rm *.zip

In [10]:
#main_path = "../input/chest-xray-pneumonia/chest_xray/"
path = "/content/gdrive/MyDrive/Kaggle/chest_xray/"

In [11]:
# from distutils.dir_util import copy_tree

# copy_tree(main_path,'temp')

In [12]:
path += os.listdir()[1]

In [13]:
path

'/content/gdrive/MyDrive/Kaggle/chest_xray/chest_xray'

In [14]:
train_path = os.path.join(path,"train")
test_path=os.path.join(path,"test")
val_path=os.path.join(path,"val")

In [16]:
# for i in ['/NORMAL/','/PNEUMONIA/']:
#     for img in os.listdir(train_path+i)[:192]:
#         os.replace(train_path+i+img, val_path+i+img)

In [25]:
val_path

'/content/gdrive/MyDrive/Kaggle/chest_xray/chest_xray/val'

In [26]:
len(os.listdir(val_path+'/NORMAL'))

200

In [27]:
len(os.listdir(val_path+'/PNEUMONIA'))

200

In [19]:
# pneumonia_train_images = glob.glob(train_path+"/PNEUMONIA/*.jpeg")
# normal_train_images = glob.glob(train_path+"/NORMAL/*.jpeg")

# pneumonia_val_images = glob.glob(val_path+"/PNEUMONIA/*.jpeg")
# normal_val_images = glob.glob(val_path+"/NORMAL/*.jpeg")

# pneumonia_test_images = glob.glob(test_path+"/PNEUMONIA/*.jpeg")
# normal_test_images = glob.glob(test_path+"/NORMAL/*.jpeg")

In [20]:
# data = pd.DataFrame(np.concatenate([[0]*len(normal_train_images) , [1] *  len(pneumonia_train_images)]),columns=["class"])

In [21]:
# sns.countplot(data['class'],data=data)

In [22]:
# define the type of augmentation techniques we will apply.
train_Datagen = ImageDataGenerator(
    rescale =1/255,
#     shear_range=10,
    zoom_range = 0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     rotation_range=20,
    fill_mode = 'nearest',
)
val_datagen = ImageDataGenerator(
    rescale =1/255,
#     shear_range=10,
#     zoom_range = 0.2,
#     horizontal_flip = True,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     rotation_range=20,
#     fill_mode = 'nearest',
)

In [23]:
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation = 'relu',kernel_regularizer= keras.regularizers.l2(l2=0.05)))
model.add(Dropout(0.7))
model.add(Dense(64, activation='relu',kernel_regularizer=keras.regularizers.l2(l2=0.05)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, None, None, 2048)  23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6

In [29]:
train_generator=train_Datagen.flow_from_directory(
    train_path,
    target_size=(150,150),
    batch_size= 32,
    class_mode='binary'
)
validation_generator = val_datagen.flow_from_directory(
        test_path,
        target_size=(150,150),
        batch_size=32,
        class_mode='binary'
)
test_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

Found 4834 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 398 images belonging to 2 classes.


In [31]:
from keras.optimizers import Adam

initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=750,
    decay_rate=0.9,
    staircase=True)

model.compile(optimizer= Adam(lr_schedule), loss='binary_crossentropy', metrics=["accuracy"])

#callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

history = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch = 4834 // 32,
                    validation_data = validation_generator,
                    validation_steps = 624 // 32)

Epoch 1/10
151/151 [==============================] - 1081s 7s/step - loss: 10.8261 - accuracy: 0.8397 - val_loss: 1.5007 - val_accuracy: 0.7862
Epoch 2/10
151/151 [==============================] - 77s 512ms/step - loss: 0.9120 - accuracy: 0.8732 - val_loss: 0.7169 - val_accuracy: 0.8470
Epoch 3/10
151/151 [==============================] - 77s 513ms/step - loss: 0.5672 - accuracy: 0.8744 - val_loss: 0.6092 - val_accuracy: 0.8322
Epoch 4/10
151/151 [==============================] - 77s 509ms/step - loss: 0.4637 - accuracy: 0.8859 - val_loss: 0.5471 - val_accuracy: 0.8109
Epoch 5/10
151/151 [==============================] - 76s 506ms/step - loss: 0.4470 - accuracy: 0.8786 - val_loss: 0.5337 - val_accuracy: 0.8257
Epoch 6/10
151/151 [==============================] - 76s 505ms/step - loss: 0.4176 - accuracy: 0.8803 - val_loss: 0.6202 - val_accuracy: 0.7664
Epoch 7/10
151/151 [==============================] - 77s 508ms/step - loss: 0.4118 - accuracy: 0.8865 - val_loss: 0.4742 - val_ac

In [32]:
model.evaluate_generator(test_generator,16)[1]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


0.9522613286972046

In [33]:
model.evaluate_generator(validation_generator,16)[1]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


0.849609375

In [34]:
# base_model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False)

# for layer in base_model.layers:
#     layer.trainable = True
# keras.regularizers.l2(l2=0.03)    
    
# model = Sequential()
# model.add(base_model)
# model.add(GlobalAveragePooling2D())
# model.add(Dense(1024, activation = 'relu',kernel_regularizer='l2'))
# model.add(Dropout(0.5))
# model.add(Dense(512, activation = 'relu', kernel_regularizer='l2'))
# model.add(Dropout(0.5))
# model.add(Dense(128, activation = 'relu' , kernel_regularizer='l2'))
# model.add(Dropout(0.7))
# model.add(Dense(1, activation='sigmoid'))
# model.summary()
        

In [35]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 pool1_pad
4 pool1_pool
5 conv2_block1_preact_bn
6 conv2_block1_preact_relu
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_pad
11 conv2_block1_2_conv
12 conv2_block1_2_bn
13 conv2_block1_2_relu
14 conv2_block1_0_conv
15 conv2_block1_3_conv
16 conv2_block1_out
17 conv2_block2_preact_bn
18 conv2_block2_preact_relu
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_pad
23 conv2_block2_2_conv
24 conv2_block2_2_bn
25 conv2_block2_2_relu
26 conv2_block2_3_conv
27 conv2_block2_out
28 conv2_block3_preact_bn
29 conv2_block3_preact_relu
30 conv2_block3_1_conv
31 conv2_block3_1_bn
32 conv2_block3_1_relu
33 conv2_block3_2_pad
34 conv2_block3_2_conv
35 conv2_block3_2_bn
36 conv2_block3_2_relu
37 max_pooling2d
38 conv2_block3_3_conv
39 conv2_block3_out
40 conv3_block1_preact_bn
41 conv3_block1_preact_relu
42 conv3_block1_1_conv
43 conv3_block1_1_bn
44 conv3_block1_1_relu
45 conv3_block1_2_pad
46 

In [36]:
for layer in base_model.layers[:165]:
    layer.trainable = False
for layer in base_model.layers[165:]:
    layer.trainable = True

In [37]:
initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=750,
    decay_rate=0.9,
    staircase=True)
model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule),  # Very slow learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [38]:
history_new = model.fit(train_generator,
                              epochs=50,
                              steps_per_epoch = 4834 // 32,
                              validation_data = validation_generator,
                              validation_steps = 624 // 32)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4982: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


151/151 [==============================] - 87s 525ms/step - loss: 0.2954 - binary_accuracy: 0.9182 - val_loss: 0.6975 - val_binary_accuracy: 0.9278
Epoch 2/50
151/151 [==============================] - 78s 519ms/step - loss: 0.2122 - binary_accuracy: 0.9322 - val_loss: 0.5263 - val_binary_accuracy: 0.9365
Epoch 3/50
151/151 [==============================] - 79s 519ms/step - loss: 0.1720 - binary_accuracy: 0.9396 - val_loss: 0.2759 - val_binary_accuracy: 0.9426
Epoch 4/50
151/151 [==============================] - 78s 518ms/step - loss: 0.1497 - binary_accuracy: 0.9452 - val_loss: 0.3106 - val_binary_accuracy: 0.9468
Epoch 5/50
151/151 [==============================] - 78s 518ms/step - loss: 0.1408 - binary_accuracy: 0.9484 - val_loss: 0.3150 - val_binary_accuracy: 0.9497
Epoch 6/50
151/151 [==============================] - 78s 517ms/step - loss: 0.1262 - binary_accuracy: 0.9509 - val_loss: 0.3362 - val_binary_accuracy: 0.9522
Epoch 7/50
151/151 [==============================] - 78s

In [39]:
model.evaluate_generator(validation_generator)[1]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


0.9752039909362793

In [40]:
model.evaluate_generator(test_generator)[1]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


0.9751599431037903